In [11]:
import pandas as pd

Parse the original xlsx file and convert it to csv. The original file can be converted from https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Haushalte-Familien/Tabellen/sonderauswertung-haushalte-regiostar.html.

Read dataset, taking only the worksheet "RegioStarStadt_Land_regioSt2017". 
- Skip initial lines that contain no info.
- Drop last four lines that contain no info.

In [12]:
filepath = "sonderauswertung-haushalte-regiostar.xlsx"
hh_df = pd.read_excel(filepath, sheet_name="RegioStarStadt_Land_regioSt2017", header=[3, 4], skiprows=[5])
hh_df = hh_df.drop(hh_df.index.values[-4:])

Create a column called "land" to store the name of the Länder.

In [13]:
hh_df[("RegioStar-Stadt-Land-Regionen", "land")] = "Land"

Sort and rename columns to increase readability.

In [14]:
hh_df = hh_df.sort_index(axis=1)
hh_df = hh_df.rename(columns = {
    "RegioStar-Stadt-Land-Regionen": "location",
    "Unnamed: 0_level_1": "type",
    "Haushalte am Haupt- und Nebenwohnsitz": "haupt_und_nebenwohnsitz",
    "Haushalte am Hauptwohnsitz": "hauptwohnsitz",
    "Insgesamt": "tot",
    "Einpersonenhaushalte": "one_person",
    "Mit zwei Personen": "two_persons",
    "Mit drei Personen": "three_persons",
    "Mit vier Personen": "four_persons",
    "Mit fünf Personen und mehr": "five_plus_persons",
})

Find rows that contain "Länder" and save their index.

In [15]:
rows_with_lands = []
for idx, row in hh_df.iterrows():
    if (type(row.iloc[1]) == str) and (row.iloc[1] != "X"):
        rows_with_lands.append((idx, row.iloc[1]))
rows_with_lands

[(0, 'Deutschland'),
 (8, 'Früheres Bundesgebiet ohne Berlin'),
 (16, 'Baden-Württemberg'),
 (24, 'Bayern'),
 (32, 'Bremen'),
 (40, 'Hamburg'),
 (48, 'Hessen'),
 (56, 'Niedersachsen'),
 (64, 'Nordrhein-Westfalen'),
 (72, 'Rheinland-Pfalz'),
 (80, 'Saarland'),
 (88, 'Schleswig-Holstein'),
 (96, 'Neue Länder einschl. Berlin'),
 (104, 'Berlin'),
 (112, 'Brandenburg'),
 (120, 'Mecklenburg-Vorpommern'),
 (128, 'Sachsen'),
 (136, 'Sachsen-Anhalt'),
 (144, 'Thüringen')]

Drop everything that is not a "Bundesland" (aggregated values can be computed from the rest)

In [16]:
hh_df = hh_df.drop(range(0, rows_with_lands[2][0]))

Fill column "land" with the right name

In [17]:
for j in range(2, 19):
    start = rows_with_lands[j][0] + 1
    if j == 18:
        end = len(hh_df) + rows_with_lands[2][0]
    else:
        end = rows_with_lands[j+1][0]
    hh_df.loc[range(start, end), ("location", "land")] = rows_with_lands[j][1]

Now drop rows that initially contained "land" names and set the new column "land" as index.

In [18]:
hh_df = hh_df.drop([el[0] for el in rows_with_lands[2:]])
hh_df = hh_df.set_index([("location", "land"), ("location", "type")])
hh_df

haupt_und_nebenwohnsitz  \
                                                                       one_person   
(location, land)  (location, type)                                                  
Baden-Württemberg Insgesamt                                                  2066   
                  Stadtregion                                                1332   
                  Metropolitane Stadtregion (MPS)                             747   
                  Regiopolitane Stadtregion (RPS)                             585   
                  Ländliche Region                                            734   
...                                                                           ...   
Thüringen         Metropolitane Stadtregion (MPS)                               X   
                  Regiopolitane Stadtregion (RPS)                             150   
                  Ländliche Region                                            306   
                  Stadtregionsnahe ländliche Region (SLR)                     140   
                  Periphere ländliche Region (PLR)                            166   

                                                                               \
                                                            tot three_persons   
(location, land)  (location, type)                                              
Baden-Württemberg Insgesamt                                5278           661   
                  Stadtregion                              3216           390   
                  Metropolitane Stadtregion (MPS)          1775           215   
                  Regiopolitane Stadtregion (RPS)          1442           175   
                  Ländliche Region                         2062           271   
...                                                         ...           ...   
Thüringen         Metropolitane Stadtregion (MPS)             X             X   
                  Regiopolitane Stadtregion (RPS)           332            39   
                  Ländliche Region                          770           102   
                  Stadtregionsnahe ländliche Region (SLR)   354            47   
                  Periphere ländliche Region (PLR)          416            55   

                                                                             \
                                                          five_plus_persons   
(location, land)  (location, type)                                            
Baden-Württemberg Insgesamt                                             225   
                  Stadtregion                                           122   
                  Metropolitane Stadtregion (MPS)                        68   
                  Regiopolitane Stadtregion (RPS)                        54   
                  Ländliche Region                                      104   
...                                                                     ...   
Thüringen         Metropolitane Stadtregion (MPS)                         X   
                  Regiopolitane Stadtregion (RPS)                         7   
                  Ländliche Region                                       19   
                  Stadtregionsnahe ländliche Region (SLR)                 9   
                  Periphere ländliche Region (PLR)                       10   

                                                                        \
                                                          four_persons   
(location, land)  (location, type)                                       
Baden-Württemberg Insgesamt                                        587   
                  Stadtregion                                      337   
                  Metropolitane Stadtregion (MPS)                  183   
                  Regiopolitane Stadtregion (RPS)                  154   
                  Ländliche Region                                 250   
...                              

Save to csv

In [19]:
hh_df.to_csv("household_land.csv")

Test loading

In [20]:
hh_df = pd.read_csv("household_land.csv", header=[0, 1], index_col=[0, 1])
hh_df

haupt_und_nebenwohnsitz  \
                                                                          one_person   
('location', 'land') ('location', 'type')                                              
Baden-Württemberg    Insgesamt                                                  2066   
                     Stadtregion                                                1332   
                     Metropolitane Stadtregion (MPS)                             747   
                     Regiopolitane Stadtregion (RPS)                             585   
                     Ländliche Region                                            734   
...                                                                              ...   
Thüringen            Metropolitane Stadtregion (MPS)                               X   
                     Regiopolitane Stadtregion (RPS)                             150   
                     Ländliche Region                                            306   
                     Stadtregionsnahe ländliche Region (SLR)                     140   
                     Periphere ländliche Region (PLR)                            166   

                                                                    \
                                                               tot   
('location', 'land') ('location', 'type')                            
Baden-Württemberg    Insgesamt                                5278   
                     Stadtregion                              3216   
                     Metropolitane Stadtregion (MPS)          1775   
                     Regiopolitane Stadtregion (RPS)          1442   
                     Ländliche Region                         2062   
...                                                            ...   
Thüringen            Metropolitane Stadtregion (MPS)             X   
                     Regiopolitane Stadtregion (RPS)           332   
                     Ländliche Region                          770   
                     Stadtregionsnahe ländliche Region (SLR)   354   
                     Periphere ländliche Region (PLR)          416   

                                                                            \
                                                             three_persons   
('location', 'land') ('location', 'type')                                    
Baden-Württemberg    Insgesamt                                         661   
                     Stadtregion                                       390   
                     Metropolitane Stadtregion (MPS)                   215   
                     Regiopolitane Stadtregion (RPS)                   175   
                     Ländliche Region                                  271   
...                                                                    ...   
Thüringen            Metropolitane Stadtregion (MPS)                     X   
                     Regiopolitane Stadtregion (RPS)                    39   
                     Ländliche Region                                  102   
                     Stadtregionsnahe ländliche Region (SLR)            47   
                     Periphere ländliche Region (PLR)                   55   

                                                                                \
                                                             five_plus_persons   
('location', 'land') ('location', 'type')                                        
Baden-Württemberg    Insgesamt                                             225   
                     Stadtregion                                           122   
                     Metropolitane Stadtregion (MPS)                        68   
                     Regiopolitane Stadtregion (RPS)                        54   
                     Ländliche Region                                      104   
...                                                                        ...   
Thüringen       